In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from numerai_blocks import *

# Numerai Blocks

> Tools for solid Numerai pipelines

This file will become your README and also the index of your documentation.

## Install

`pip install numerai-blocks`

## How to use

Development notebooks are in the `dev_nbs` directory. Example and educational notebooks can be found in the `edu_nbs` directory.

1. Downloaders
2. Dataloaders
3. Dataset objects (with arbitrary metadata)
4. Preprocessors and Postprocessors
5. Model, ModelPipeline and ModelPipelineCollection (Or FeatureUnion??)
6. Predictions (with arbitrary metadata)
7. Evaluators
8. Submittors
9. Key (containing authentication info)

In [ ]:
1+1

2

In [ ]:
import uuid
print(uuid.uuid4())

649ce02a-6fb4-4b1f-8375-d3b46ed391c9


In [ ]:
from numerai_blocks.download import Check
ch = Check()
ch.one

1

## Contributing

After you clone this repository, please run `nbdev_install_git_hooks` in your terminal. This sets up git hooks, which clean up the notebooks to remove the extraneous stuff stored in the notebooks (e.g. which cells you ran) which causes unnecessary merge conflicts.

### Branch structure


In [ ]:
#hide
from rich.console import Console
from rich.tree import Tree

console = Console(record=True, width=100)

tree = Tree("Brranch structure", guide_style="bold bright_black")

main_tree = tree.add("📦 main (release)", guide_style="bright_black")
dev_tree = main_tree.add("👨‍💻 dev")
feature_tree = dev_tree.add(":sparkles: feature/1")
dev_tree.add(":sparkles: feature/2")
dev_tree.add(":sparkles: feature/3");

In [ ]:
console.print(tree)

Brranch structure                                                                                   
┗━━ 📦 main                                                                                         
    ┗━━ 👨‍💻 dev                                                                                    
        ┣━━ ✨ feature/1                                                                            
        ┣━━ ✨ feature/2                                                                            
        ┗━━ ✨ feature/3

In [ ]:
#hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script; notebook2script()